In [1]:
import pdfplumber
import json
from openai import OpenAI
import os

In [2]:
GPT_API_KEY = os.getenv("OPENAI_API_KEY")

In [3]:
# Inicializa el cliente
client = OpenAI(api_key=GPT_API_KEY)

In [4]:
def leer_texto_pdf(path_pdf):
    with pdfplumber.open(path_pdf) as pdf:
        texto = "\n".join([pagina.extract_text() for pagina in pdf.pages if pagina.extract_text()])
    return texto

In [5]:
def construir_prompt(texto_factura):
    prompt = f"""
Extrae los siguientes campos de esta factura:

- Nombre del emisor
- RFC del emisor
- Fecha de emisión
- Nombre del receptor
- RFC del receptor
- Subtotal
- IVA
- Total

Factura:
---
{texto_factura}
---
Responde solo en formato JSON válido y sin ningún comentario adicional.
    """
    return prompt

In [6]:
def extraer_datos_con_gpt(path_pdf):
    texto = leer_texto_pdf(path_pdf)
    prompt = construir_prompt(texto)

    respuesta = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )

    contenido = respuesta.choices[0].message.content

    try:
        return json.loads(contenido)
    except Exception as e:
        print("⚠️ Error al interpretar la respuesta como JSON.")
        print("Respuesta cruda:\n", contenido)
        return None

In [7]:
resultado = extraer_datos_con_gpt(r"C:\Users\josei\fr8-projects\GitHub\fr8-invoices-information\data\raw\unique\Estadía 53572-1.pdf")
print(json.dumps(resultado, indent=2, ensure_ascii=False))

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


⚠️ Error al interpretar la respuesta como JSON.
Respuesta cruda:
 ```json
{
  "Nombre del emisor": "ITZEL ESTEFANIA HERNANDEZ RAMIREZ",
  "RFC del emisor": "HERI910707B41",
  "Fecha de emisión": "2024-02-14T08:58:41",
  "Nombre del receptor": "FREIGHT APP DE MEXICO",
  "RFC del receptor": "FHM190118EN7",
  "Subtotal": "7,800.00 MXN",
  "IVA": "1,248.00 MXN",
  "Total": "9,048.00 MXN"
}
```
null
